# Lab 2: Models và SQL Transformations - Tạo dbt Models

## 🎯 Objectives
- Hiểu dbt models là gì
- Tạo staging models từ raw data
- Tạo marts models với aggregations
- Sử dụng Jinja templating trong SQL
- Hiểu model dependencies và ref()
- Sử dụng sources và source freshness
- Hiểu các materializations (view, table, incremental)

## 📋 Prerequisites
- Hoàn thành Lab 1
- PostgreSQL đang chạy
- Sample data đã được load

## 🏗️ dbt Models Overview

**dbt Models** là SQL files trong thư mục `models/`:
- Mỗi `.sql` file = một dbt model
- Models có thể reference nhau bằng `ref()`
- Models có thể reference sources bằng `source()`
- Models được compile thành SQL và chạy trong database

### Model Layers:
```
Raw Data → Staging → Intermediate → Marts
   ↓         ↓           ↓            ↓
Sources   Clean      Transform    Analytics
```


## 1. Staging Models - Clean Raw Data

Staging models clean và standardize data từ raw sources.


In [ ]:
# Xem staging model example
from pathlib import Path

project_root = Path('.').resolve()
if project_root.name != 'dbt_lab':
    project_root = project_root.parent / 'dbt_lab'

staging_model = project_root / 'models' / 'staging' / 'stg_customers.sql'

if staging_model.exists():
    print("📄 Staging Model: stg_customers.sql")
    print("=" * 60)
    with open(staging_model, 'r') as f:
        print(f.read())
    print("=" * 60)
    
    print("\n💡 Key Points:")
    print("  - Uses {{ source('raw', 'customers') }} to reference source")
    print("  - Config block với materialized='view'")
    print("  - Tags để organize models")
    print("  - Adds dbt_loaded_at timestamp")
else:
    print(f"⚠️  File not found: {staging_model}")


## 2. Marts Models - Analytics Layer

Marts models tạo final analytics tables với aggregations và business logic.


In [ ]:
# Xem marts model example
marts_model = project_root / 'models' / 'marts' / 'customer_orders.sql'

if marts_model.exists():
    print("📄 Marts Model: customer_orders.sql")
    print("=" * 60)
    with open(marts_model, 'r') as f:
        print(f.read())
    print("=" * 60)
    
    print("\n💡 Key Points:")
    print("  - Uses {{ ref('stg_customers') }} và {{ ref('stg_orders') }}")
    print("  - Materialized as 'table' (not view)")
    print("  - Aggregates order data per customer")
    print("  - Calculates lifetime_value và order_count")
else:
    print(f"⚠️  File not found: {marts_model}")


## 3. Jinja Templating trong SQL

dbt sử dụng Jinja templating để làm SQL dynamic và reusable.


In [ ]:
print("🔧 Jinja Templating Examples:")
print("=" * 60)
print("""
1. Reference other models:
   {{ ref('stg_customers') }}
   {{ ref('staging', 'stg_customers') }}

2. Reference sources:
   {{ source('raw', 'customers') }}

3. Variables:
   {{ var('start_date') }}
   {{ var('end_date', '2024-12-31') }}  # với default value

4. Conditionals:
   {% if var('include_test_data', false) %}
   select * from ...
   {% else %}
   select * from ... where is_test = false
   {% endif %}

5. Loops:
   {% for column in ['customer_id', 'order_id', 'product_id'] %}
   {{ column }},
   {% endfor %}

6. Macros:
   {{ cents_to_dollars('amount_cents') }}
   {{ generate_surrogate_key(['customer_id', 'order_date']) }}

7. Config:
   {{ config(materialized='table', tags=['marts']) }}

8. This (current model):
   {{ this }}
   {{ this.schema }}
   {{ this.name }}
""")
print("=" * 60)


## 4. Run dbt Models

Chạy models với `dbt run`.


In [ ]:
# Run dbt models
import subprocess
import os

def run_dbt_models(select=None):
    """Run dbt models"""
    print("🚀 Running dbt models...")
    print("=" * 60)
    
    original_dir = os.getcwd()
    try:
        os.chdir(project_root)
        
        cmd = ['dbt', 'run', '--profiles-dir', '.', '--project-dir', '.']
        if select:
            cmd.extend(['--select', select])
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=60
        )
        
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ Models ran successfully!")
        else:
            print("\n❌ Models failed!")
            print("\nError output:")
            print(result.stderr)
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
    finally:
        os.chdir(original_dir)

# Uncomment để chạy:
# run_dbt_models()
# run_dbt_models('staging.*')  # Chỉ chạy staging models
# run_dbt_models('customer_orders')  # Chỉ chạy một model


In [ ]:
print("📊 Materialization Types:")
print("=" * 60)
print("""
1. view (default)
   - Tạo SQL view trong database
   - Fastest to build, slowest to query
   - Good cho: Staging models, frequently changing models
   - Config: {{ config(materialized='view') }}

2. table
   - Tạo physical table trong database
   - Slowest to build, fastest to query
   - Good cho: Final marts, frequently queried models
   - Config: {{ config(materialized='table') }}

3. incremental
   - Chỉ insert/update new rows
   - Efficient cho large tables
   - Requires unique_key và strategy
   - Config: 
     {{ config(
         materialized='incremental',
         unique_key='order_id',
         on_schema_change='fail'
     ) }}

4. ephemeral
   - Không tạo object trong database
   - Inline vào models khác
   - Good cho: Intermediate calculations
   - Config: {{ config(materialized='ephemeral') }}

5. snapshot
   - Track changes over time
   - Good cho: Slowly changing dimensions
   - Config: {{ config(materialized='snapshot') }}
""")
print("=" * 60)


## 6. Compile SQL để xem Generated Code

Sử dụng `dbt compile` để xem SQL được generate.


In [ ]:
# Compile dbt models
def compile_dbt_models():
    """Compile dbt models để xem generated SQL"""
    print("🔨 Compiling dbt models...")
    print("=" * 60)
    
    original_dir = os.getcwd()
    try:
        os.chdir(project_root)
        
        result = subprocess.run(
            ['dbt', 'compile', '--profiles-dir', '.', '--project-dir', '.'],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ Models compiled successfully!")
            print("\n💡 Check target/compiled/ để xem generated SQL")
        else:
            print("\n❌ Compilation failed!")
            print(result.stderr)
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
    finally:
        os.chdir(original_dir)

# Uncomment để compile:
# compile_dbt_models()


## 7. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Staging models để clean raw data
2. Marts models cho analytics
3. Jinja templating trong SQL
4. Model dependencies với ref()
5. Sources với source()
6. Materializations (view, table, incremental)
7. Compile và run models

### 📚 Next Lab:
- **Lab 3**: Testing và Documentation
- Data quality tests
- Generic tests
- Custom tests
- Documentation

### 💡 Key Takeaways:

**Model Layers:**
- Staging: Clean raw data
- Intermediate: Transformations
- Marts: Final analytics tables

**Jinja Functions:**
- `ref()`: Reference other models
- `source()`: Reference sources
- `var()`: Use variables
- `config()`: Configure models

**Materializations:**
- `view`: Fast build, slow query
- `table`: Slow build, fast query
- `incremental`: Efficient cho large tables

### 🔗 Useful Links:
- [dbt Models](https://docs.getdbt.com/docs/build/models)
- [Jinja](https://docs.getdbt.com/docs/build/jinja-macros)
- [Materializations](https://docs.getdbt.com/docs/build/materializations)
